In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

warnings.filterwarnings('ignore')
plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 품목: 배추

In [3]:
#데이터 불러오기
cab = pd.read_csv("C:/Users/keogu/Desktop/빅데이터 자료분석/new data/배추.csv",parse_dates=['SALEDATE'])

In [4]:
cab.head()

,PUM_NM,LV_NM,TOT_AMT,SAN_NM,SALEDATE,CMP_NM,DAN_NM,WHSAL_NM,SIZE_NM,COST,POJ_NM,TOT_QTY,QTY,KIND_NM,DANQ
0,배추,특,80000.0,경기도 광주시,2014-01-03,한국청과,kg,서울가락도매,.,3200.0,상자,25.0,25.0,쌈배추,1.0
1,배추,특,77500.0,경기도 광주시,2014-01-03,한국청과,kg,서울가락도매,.,3100.0,상자,25.0,25.0,쌈배추,1.0
2,배추,특,75000.0,경기도 광주시,2014-01-03,한국청과,kg,서울가락도매,.,3000.0,상자,25.0,25.0,쌈배추,1.0
3,배추,특,145700.0,서울특별시,2014-01-03,한국청과,kg,서울가락도매,.,4700.0,상자,31.0,31.0,쌈배추,1.0
4,배추,특,138000.0,서울특별시,2014-01-03,한국청과,kg,서울가락도매,.,4600.0,상자,30.0,30.0,쌈배추,1.0


In [5]:
cab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2784235 entries, 0 to 2784234
Data columns (total 15 columns):
 #   Column    Dtype         
---  ------    -----         
 0   PUM_NM    object        
 1   LV_NM     object        
 2   TOT_AMT   float64       
 3   SAN_NM    object        
 4   SALEDATE  datetime64[ns]
 5   CMP_NM    object        
 6   DAN_NM    object        
 7   WHSAL_NM  object        
 8   SIZE_NM   object        
 9   COST      float64       
 10  POJ_NM    object        
 11  TOT_QTY   float64       
 12  QTY       float64       
 13  KIND_NM   object        
 14  DANQ      float64       
dtypes: datetime64[ns](1), float64(5), object(9)
memory usage: 318.6+ MB


# 전처리

## 필요없는 컬럼 삭제  
**필요없는 칼럼**
- PUM_NM: 품목
- LV_NM: 등급  
- CMP_NM: 법인
- DAN_NM: 단위
- WHSAL_NM: 도매시장 
- SIZE_NM: 크기
- COST: 단가
- POJ_NM: 포장 
- QTY: 물량
- KIND_NM: 품종 
- DANQ: 단위물량

In [6]:
# 필요없는 칼럼 삭제
cab.drop(columns=['CMP_NM','WHSAL_NM','POJ_NM','KIND_NM','PUM_NM','LV_NM','SIZE_NM','QTY','DANQ','COST','DAN_NM'], axis=1, inplace=True)

**남은 컬럼**
- TOT_AMT: 총 금액
- SAN_NM: 산지
- SALEDATE: 경락 일자   
- TOT_QTY:총 물량

## 총 금액 / 총 물량 

### 거래 취소 데이터 제거  
- 총 금액, 총 물량 음수거나 0인 데이터 제거

In [7]:
# 43406건
cab[(cab['TOT_AMT'] <= 0)|(cab['TOT_QTY'] <= 0)]

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY
288,-2952000.0,전라남도 해남군,2014-01-03,-7200.0
291,-442800.0,전라남도 해남군,2014-01-03,-1800.0
297,0.0,전라남도 해남군,2014-01-04,1800.0
298,0.0,전라남도 해남군,2014-01-04,7200.0
355,-199500.0,전라남도 해남군,2014-01-04,-700.0
...,...,...,...,...
2783518,0.0,경기도 연천군,2021-10-30,30.0
2783521,0.0,경기도 연천군,2021-10-30,20.0
2783589,0.0,경상북도 문경시,2021-10-30,50.0
2783593,0.0,경상북도 문경시,2021-10-30,20.0


In [8]:
#제거
drop_index = cab[(cab['TOT_AMT'] <= 0)|(cab['TOT_QTY'] <= 0)].index
cab.drop(index = drop_index, axis=0, inplace=True)

### 이상치 제거  

In [9]:
#단위 당 가격 칼럼 생성
cab['PER_PRICE'] = cab['TOT_AMT']/cab['TOT_QTY']

In [10]:
cab

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY,PER_PRICE
0,80000.0,경기도 광주시,2014-01-03,25.0,3200.0
1,77500.0,경기도 광주시,2014-01-03,25.0,3100.0
2,75000.0,경기도 광주시,2014-01-03,25.0,3000.0
3,145700.0,서울특별시,2014-01-03,31.0,4700.0
4,138000.0,서울특별시,2014-01-03,30.0,4600.0
...,...,...,...,...,...
2784230,114000.0,충남 부여군,2021-10-30,24.0,4750.0
2784231,8750.0,경남 밀양시,2021-10-30,5.0,1750.0
2784232,140000.0,경남 밀양시,2021-10-30,80.0,1750.0
2784233,34000.0,경남 밀양시,2021-10-30,8.0,4250.0


In [11]:
cab.loc[(cab['PER_PRICE'] < cab['PER_PRICE'].quantile(0.025)) | (cab['PER_PRICE'] > cab['PER_PRICE'].quantile(0.975))]

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY,PER_PRICE
651,30000.0,경기도 평택시,2014-01-06,240.0,125.000000
800,302400.0,전라남도 해남군,2014-01-06,2800.0,108.000000
807,237600.0,전라남도 해남군,2014-01-06,2200.0,108.000000
925,350000.0,전라남도 해남군,2014-01-07,3000.0,116.666667
946,49400.0,경기도 고양시,2014-01-07,456.0,108.333333
...,...,...,...,...,...
2784061,105000.0,전라남도 무안군,2021-10-30,1050.0,100.000000
2784062,105000.0,전라남도 무안군,2021-10-30,1050.0,100.000000
2784063,126000.0,전라남도 무안군,2021-10-30,1050.0,120.000000
2784085,20000.0,경기 부천시 소사구,2021-10-30,2.0,10000.000000


In [12]:
drop_index_price = cab.loc[(cab['PER_PRICE'] < cab['PER_PRICE'].quantile(0.025)) | (cab['PER_PRICE'] > cab['PER_PRICE'].quantile(0.975))].index
cab.drop(index = drop_index_price, axis=0, inplace=True)

In [13]:
cab

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY,PER_PRICE
0,80000.0,경기도 광주시,2014-01-03,25.0,3200.0
1,77500.0,경기도 광주시,2014-01-03,25.0,3100.0
2,75000.0,경기도 광주시,2014-01-03,25.0,3000.0
3,145700.0,서울특별시,2014-01-03,31.0,4700.0
4,138000.0,서울특별시,2014-01-03,30.0,4600.0
...,...,...,...,...,...
2784230,114000.0,충남 부여군,2021-10-30,24.0,4750.0
2784231,8750.0,경남 밀양시,2021-10-30,5.0,1750.0
2784232,140000.0,경남 밀양시,2021-10-30,80.0,1750.0
2784233,34000.0,경남 밀양시,2021-10-30,8.0,4250.0


In [14]:
cab.describe()

,TOT_AMT,TOT_QTY,PER_PRICE
count,2.609326e+06,2.609326e+06,2.609326e+06
mean,6.463668e+05,1.020340e+03,1.323472e+03
std,1.438188e+06,1.956846e+03,1.438383e+03
min,1.000000e+02,1.000000e-01,1.300000e+02
25%,7.800000e+04,6.000000e+01,4.333333e+02
50%,2.160000e+05,3.000000e+02,8.000000e+02
75%,5.600000e+05,9.680000e+02,1.575000e+03
max,2.083360e+08,2.604200e+05,8.950000e+03


## 산지  

### 산지 null값   
- 주산지 선정에 활용, 제거하지 말고 사용

In [15]:
#산지 null값 77411건, 삭제x
cab[cab['SAN_NM'].isnull()]

,TOT_AMT,SAN_NM,SALEDATE,TOT_QTY,PER_PRICE
1286,333700.0,NaN,2014-01-08,376.0,887.5
1300,108000.0,NaN,2014-01-08,240.0,450.0
1301,108000.0,NaN,2014-01-08,240.0,450.0
1302,360000.0,NaN,2014-01-08,384.0,937.5
1303,360000.0,NaN,2014-01-08,384.0,937.5
...,...,...,...,...,...
2783254,705600.0,NaN,2021-10-30,2016.0,350.0
2783310,500000.0,NaN,2021-10-30,2000.0,250.0
2783326,3360000.0,NaN,2021-10-30,8000.0,420.0
2783363,580000.0,NaN,2021-10-30,2000.0,290.0


### 산지명 통일  
- 17개의 행정구역으로 통일  
    - 경기도 / 서울특별시 / 전라남도 / 충청남도 / 강원도 / 광주광역시 / 경상북도 / 전라북도 / 경상남도 / 충청북도 / 산지없음 / 인천광역시 / 제주도 / 대전광역시 / 부산광역시 / 세종특별자치시 / 대구광역시 / 울산광역시 / 산지없음
- 수입산은 삭제

In [16]:
# 산지명
cab['SAN_NM'].unique()

array(['경기도 광주시', '서울특별시', '전라남도 해남군', '전라남도 진도군', '충청남도 당진군', '강원도 홍천군',
       '전라남도 신안군', '경기도 고양시', '광주 서구', '경기도 평택시', '전라남도 무안군', '경기도 이천시',
       '광주 북구', '충청남도 예산군', '경상북도 김천시', '경기도 용인시', '전라북도 고창군', '경상북도 영양군',
       '경기도 양평군', '경상남도 밀양시', '경기도 하남시', '충청북도 괴산군', '서울특별시 송파구',
       '경기도 성남시', '전라남도 나주시', '경기도 파주시', '충청남도 태안군', '전라남도 영광군',
       '충청남도 부여군', '경기도 김포시', '강원도 평창군', '경상북도 구미시', '강원도 영월군',
       '광주광역시 광산구', '경기도 안성시', '전라남도 화순군', '서울특별시 양천구', '서울특별시 종로구',
       '전라남도 목포시', '전라남도 순천시', nan, '전라남도 장흥군', '강원도 춘천시', '중국',
       '경상북도 성주군', '경기도 안양시', '경상북도 경산시', '충청남도 서산시', '충청북도 진천군',
       '경기도 남양주시', '전라북도 진안군', '인천광역시 부평구', '충청북도 제천시', '전라북도 전주시',
       '전라북도 순창군', '전라남도 영암군', '전라남도 함평군', '서울 강서구', '제주도 제주시', '강원도 정선군',
       '전라남도 완도군', '충청남도 홍성군', '전라남도 장성군', '전라남도 구례군', '경기도 여주군',
       '강원도 횡성군', '광주 남구', '대전광역시 유성구', '서울특별시 동대문구', '서울특별시 중랑구',
       '경기도 포천시', '강원도 인제군', '경상남도 합천군', '제주도 서귀포시', '경상북도 청도군',
       '경기도 부천시', '충청북도 음성군', '부산광역시 사상구',

In [17]:
# 산지명 앞 글자만 분리
name_split = cab['SAN_NM'].str.split(" ")
cab['SAN_NM'] = name_split.str.get(0)

In [18]:
# 분리된 산지명
cab['SAN_NM'].unique()

array(['경기도', '서울특별시', '전라남도', '충청남도', '강원도', '광주', '경상북도', '전라북도',
       '경상남도', '충청북도', '광주광역시', nan, '중국', '인천광역시', '서울', '제주도', '대전광역시',
       '부산광역시', '세종시', '대구광역시', '대구', '인천', '강원', '경북', '충남', '경기', '대전',
       '부산', '전남', '충북', '전북', '울산', '울산광역시', '경남', '제주', '수입산', '뉴질랜드',
       '미얀마'], dtype=object)

In [19]:
# 수입산 삭제
drop_index = cab[(cab['SAN_NM'] == '수입산')|(cab['SAN_NM'] == '뉴질랜드')|(cab['SAN_NM'] == '미얀마')|(cab['SAN_NM'] == '중국')].index
cab.drop(index = drop_index, axis=0, inplace=True)

In [20]:
# 산지명 행정구역명으로 통일
cab['SAN_NM'].replace(['경기','서울','전남','충남','강원','경북','전북','경남','충북','광주','제주','대전','부산','대구','세종시','울산','인천'],
                      ['경기도','서울특별시','전라남도','충청남도','강원도','경상북도','전라북도','경상남도','충청북도','광주광역시','제주도','대전광역시','부산광역시','대구광역시','세종특별자치시','울산광역시','인천광역시'],inplace=True)

In [21]:
# null값은 산지 없음으로 변경
cab['SAN_NM'].fillna('산지없음',inplace=True)

In [ ]:
# 최종 산지명
cab['SAN_NM'].unique()

In [ ]:
print(len(cab['SAN_NM'].unique()))

### 주산지 설정

In [ ]:
# 14년 ~ 21년도 총 물량
cab.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False)/sum(cab.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False)) * 100

In [ ]:
# 주산지 비율
cab.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False)[:2]/sum(cab.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=False)[:2]) * 100

In [ ]:
# 년도별 물량
for i in (2014,2015,2016,2017,2018,2019,2020,2021):
    cab_year = cab[cab['SALEDATE'].dt.year == i]
    joosan = cab_year.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=True).index[-1]
    joosan_amt = cab_year.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=True)[-1]
    ratio = joosan_amt / sum(cab_year.groupby(['SAN_NM'])['TOT_QTY'].sum().sort_values(ascending=True)) * 100
    print(i,'년도', '\n주산지:', joosan, '\n총 물량:', joosan_amt, '\n비율:', round(ratio,2))

In [ ]:
# 년도별 주산지 시각화
specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}],[{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=4, cols=2, specs=specs,
                    subplot_titles=['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021'])

fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2014].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2014].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2014',), 1, 1)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2015].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2015].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2015',), 1, 2)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2016].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2016].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2016',), 2, 1)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2017].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2017].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2017',), 2, 2)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2018].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2018].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2018',), 3, 1)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2019].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2019].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2019',), 3, 2)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2020].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2020].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2020',), 4, 1)
fig.add_trace(go.Pie(labels=cab[cab['SALEDATE'].dt.year == 2021].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['SAN_NM'], 
                     values=cab[cab['SALEDATE'].dt.year == 2021].groupby(['SAN_NM'])['TOT_QTY'].sum().reset_index()['TOT_QTY'],
                     name='2021',), 4, 2)

fig.update_traces(hole=.4, hoverinfo='label+percent+name', textinfo='none')
fig.update(layout_title_text='년도별 주산지 비율',
           layout_showlegend=True)

fig = go.Figure(fig)
fig.show()

In [ ]:
# 산지, 단위 당 가격 제거 후 주 단위로 변환
cab.drop(['SAN_NM'],axis=1,inplace=True)

## 주별로 묶기

In [29]:
cab.head()

,TOT_AMT,SALEDATE,TOT_QTY
0,80000.0,2014-01-03,25.0
1,77500.0,2014-01-03,25.0
2,75000.0,2014-01-03,25.0
3,145700.0,2014-01-03,31.0
4,138000.0,2014-01-03,30.0


In [30]:
# 날짜 인덱스 설정
cab.set_index(['SALEDATE'], inplace=True)

In [31]:
# 주 단위로 변환(월요일 기준)
cab_week= cab.resample(rule='W').sum()

In [32]:
cab_week.head()

,TOT_AMT,TOT_QTY
SALEDATE,,
2014-01-05,435201450.0,956813.0
2014-01-12,946992860.0,2429296.0
2014-01-19,814706210.0,2315261.0
2014-01-26,952137550.0,2809657.0
2014-02-02,481964200.0,1201776.0


In [33]:
# 단위 당 가격 설정
cab_week['PER_PRICE'] = cab_week['TOT_AMT']/cab_week['TOT_QTY']

In [37]:
# 주 단위 데이터
cab_week

,TOT_QTY,PER_PRICE
SALEDATE,,
2014-01-05,956813.00,454.844834
2014-01-12,2429296.00,389.821932
2014-01-19,2315261.00,351.885256
2014-01-26,2809657.00,338.880351
2014-02-02,1201776.00,401.043289
...,...,...
2021-10-03,7902093.30,603.304145
2021-10-10,7974921.81,499.519438
2021-10-17,8334563.75,384.797507


In [35]:
# 총 가격 제거
cab_week.drop(['TOT_AMT'],axis=1,inplace=True)

## 데이터 저장

In [36]:
cab_week.to_csv("C:/Users/keogu/Desktop/빅데이터 자료분석/new data/배추_전처리.csv")